In [2]:
from filter_datasets import *
import json
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa

## 1. Get MassIVE

In [14]:
databases_massIVE = getAllDts() #get all datesets available on massIVE
MassIVE_datasets_df = pd.DataFrame(databases_massIVE)
MassIVE_datasets_df.species[MassIVE_datasets_df.species.str.contains('3702')] # Arabidopsis positive control. Search made through taxon ID
MassIVE_datasets_df.head()

,dataset,datasetNum,title,user,site,flowname,createdMillis,created,description,fileCount,...,keywords,pi,complete,quant_analysis,status,private,hash,px,task,id
0,MSV000088303,88303,GNPS - DDA analysis on leaf extracts of 6 Aste...,kbkang,MassIVE,MASSIVE-COMPLETE,1635921029000,"Nov. 2, 2021, 11:30 PM",LC-Q/TOF DDA analysis on leaf extracts of 6 As...,10,...,Aster;leaf,"[{'name': 'Kyo Bin Kang ', 'email': 'kbkang@so...",false,Quantification Results,Partial,false,,,bb6a7269a69749f897998354219db189,0
1,MSV000088298,88298,The landscape of the SARS-CoV-2 HLA-peptidome,ccms,MassIVE,MASSIVE-COMPLETE,1635786592000,"Nov. 1, 2021, 10:09 AM",The human leukocyte antigen (HLA)-bound-viral ...,259,...,Sars-cov-2; Immunopeptidomics; Hla-peptidomics,"[{'name': 'Prof. Yardena Samuels', 'email': 'y...",false,null,Partial,false,,PXD023614,9c26cd591151455b854e97d9f0c6cfe0,1
2,MSV000088291,88291,B metadata-massive datasets upload-2021-11-1,xuyaping_1995,MassIVE,MASSIVE-COMPLETE,1635738512000,"Oct. 31, 2021, 8:48 PM",Screening and identifying PAs of chinese medic...,10,...,"Chinese medicines, Pyrrolizidine alkaloids, no...","[{'name': 'Sunshine', 'email': 'sunshine_xyp@1...",false,null,Partial,false,,,f6028f8d889a4ce3b6cc625b8f9ddeac,2
3,MSV000088290,88290,TIMSFolding allows the use of ion mobility enh...,orsburn,MassIVE,MASSIVE-COMPLETE,1635699235000,"Oct. 31, 2021, 9:53 AM",Files associated with the TIMSFolding preprint...,41,...,TIMSTOF;Spectral libraries;DDA;Historic softwa...,"[{'name': 'Benjamin Orsburn', 'email': 'borsbu...",false,null,Partial,false,,PXD029447,f742e08a9f634f4c9fb16261bbc25b8c,3
4,MSV000088288,88288,Metabolomic data of fungi from the Brazilian A...,pwpgomes,MassIVE,MASSIVE-COMPLETE,1635624458000,"Oct. 30, 2021, 1:07 PM",Metabolomic data of fungi from the Brazilian A...,205,...,Fungi;Amazon;Brazil,"[{'name': 'Anelize Bauermeister', 'email': 'an...",false,null,Partial,false,,,abe18ab6d48b425a85f185147aff445a,4


In [15]:
MassIVE_datasets_df.columns

Index(['dataset', 'datasetNum', 'title', 'user', 'site', 'flowname',
       'createdMillis', 'created', 'description', 'fileCount', 'fileSizeKB',
       'spectra', 'psms', 'peptides', 'variants', 'proteins', 'search_psms',
       'search_peptides', 'search_variants', 'search_proteins',
       'search_spectra', 'reanalysis_psms', 'reanalysis_peptides',
       'reanalysis_variants', 'reanalysis_proteins', 'species', 'instrument',
       'modification', 'keywords', 'pi', 'complete', 'quant_analysis',
       'status', 'private', 'hash', 'px', 'task', 'id'],
      dtype='object')

In [16]:
# massive_plantid_proteomics.tsv is the result from filterMassIVE function on MassIVE databases
MassIVE_passed_df = pd.read_csv('massive_plantid_proteomics.tsv', sep='\t')
MassIVE_passed_df.head()

,dataset,datasetNum,title,user,site,flowname,createdMillis,created,description,fileCount,...,pi,complete,quant_analysis,status,private,hash,px,task,id,search
0,MSV000087822,87822,TMK-based cell surface auxin signaling activat...,haiyanzheng,MassIVE,MASSIVE-COMPLETE,1626290200000,"Jul. 14, 2021, 12:16 PM",The phytohormone auxin controls a myriad of pr...,72,...,"[{'name': 'Yang, zhenbiao', 'email': 'zhenbiao...",False,NaN,Partial,False,NaN,NaN,ab62b7b1d5e5455dac665ed8629caf97,119,0
1,MSV000087472,87472,Complement of proteins associated with clathri...,dahhan,MassIVE,MASSIVE-COMPLETE,1621570970000,"May. 20, 2021, 9:22 PM","In Eukaryotes, clathrin coated vesicles (CCVs)...",43,...,"[{'name': 'Sebastian Y. Bednarek', 'email': 's...",True,NaN,Complete,False,NaN,PXD026180,d36fa49fcec846a288df2ffd04669068,260,1
2,MSV000086794,86794,Changes in mitochondrial proteome upon RNAi kn...,minsookim,MassIVE,MASSIVE-COMPLETE,1612266144000,"Feb. 2, 2021, 3:42 AM",To investigate how reduced ATP synthase subuni...,21,...,"[{'name': 'Elizabeth Vierling', 'email': 'vier...",False,NaN,Partial,False,NaN,NaN,b9b6448f3c534d37ae178b646f45ac3b,519,1
3,MSV000086324,86324,NOT9 Co-IP in Arabidopsis thaliana,klement,MassIVE,MASSIVE-COMPLETE,1603120368000,"Oct. 19, 2020, 8:12 AM",Interacting partners of NOT9 in Arabidopsis th...,13,...,"[{'name': 'Eva Klement', 'email': 'klement.eva...",False,NaN,Partial,False,NaN,NaN,910cf186e64d4d8593e9d5eecefc6c18,732,0
4,MSV000086248,86248,SA-Induced MTD-FLAG Interactome in A. thaliana,tch21,MassIVE,MASSIVE-COMPLETE,1602159199000,"Oct. 8, 2020, 5:13 AM",Identification of the salicylic acid (SA) indu...,506,...,"[{'name': 'John Williamson', 'email': 'jdw@ncs...",True,NaN,Complete,False,NaN,PXD021892,4b0caa7982ec4bbf9bc3c7c68ad1d6a2,760,0


In [17]:
#Tried to find zmtable on massIVE datasets. 
for dataset in MassIVE_passed_df["dataset"]:  
    massive_obj = ppx.MassiveProject(
        f"{dataset}")
    massive_remote_files = massive_obj.remote_files(glob = ".mztable")
    if massive_remote_files:
        print(massive_remote_files)
    

KeyboardInterrupt: 

## 2. GET PRIDE

In [3]:
# pride_plant.json is the result from filterPride function on PRIDE databases
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total of plants retrieved from pride database: {len(pride_plant)}")

Total of plants retrieved from pride database: 1464


In [8]:
projetc_identifiers = [] # holds project number for all plants species data from pride
accession_number = [] # holds taxon id for all plants species data from pride
for key,value in pride_plant.items():
    projetc_identifiers.append(key)
    accession_number.append(value['accession'])
    

In [9]:
projetc_identifiers[:5]

['PRD000037', 'PRD000044', 'PRD000051', 'PRD000084', 'PRD000096']

In [10]:
accession_number[:5]

['3055', '3702', '4577', '3702', '3702']

In [13]:
proj = ppx.find_project(projetc_identifiers[0]) 
remote_files = proj.remote_files()
print(remote_files)

error_temp: Failed after 10 reconnect(s), the last error was: [Errno 104] Connection reset by peer

In [8]:
#Tried to check if the same error occours for other files, but takes to long and the host connection stops 
for projects in projetc_identifiers:
    try:
        proj = ppx.find_project(projects) 
        remote_files = proj.remote_files()
        print(remote_files)
    except:
        pass
        

In [18]:
findmztab(projetc_identifiers[0])==None # select m/z tables from a given project.  

error_temp: Failed after 10 reconnect(s), the last error was: [Errno 104] Connection reset by peer

In [19]:
#Search for project with mztab in pride plants database
for i in projetc_identifiers:
    if findmztab(i)!= None: #Selec only those files with m\z tables
        print(i)
        break #stops at fist result 

error_temp: Failed after 10 reconnect(s), the last error was: 550 Failed to change directory.

In [20]:
#Explores the first result
#Everything from here must be done to all findings 
proj = ppx.find_project('PRD000084')

In [21]:
proj.metadata.keys()

dict_keys(['accession', 'title', 'additionalAttributes', 'projectDescription', 'sampleProcessingProtocol', 'dataProcessingProtocol', 'projectTags', 'keywords', 'doi', 'submissionType', 'submissionDate', 'publicationDate', 'submitters', 'labPIs', 'affiliations', 'instruments', 'softwares', 'quantificationMethods', 'countries', 'sampleAttributes', 'organisms', 'organismParts', 'diseases', 'references', 'identifiedPTMStrings', '_links'])

In [22]:
pride_plant['PRD000084']

{'name': 'Arabidopsis thaliana (mouse-ear cress)', 'accession': '3702'}

In [23]:
remote_files = proj.remote_files()
print(remote_files)

error_temp: Failed after 10 reconnect(s), the last error was: 550 Failed to change directory.

In [24]:
findmztab(i)

error_temp: Failed after 10 reconnect(s), the last error was: 550 Failed to change directory.

In [25]:
downloaded = proj.download("generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz")
print(downloaded)

error_temp: Failed after 10 reconnect(s), the last error was: 550 Failed to change directory.

In [26]:
!gzip -df /home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz 

gzip: /home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab.gz: No such file or directory


In [27]:
!ls /home/tiago/.ppx/PRD000084/generated/

ls: cannot access '/home/tiago/.ppx/PRD000084/generated/': No such file or directory


In [28]:
tables = mztab.MzTab("/home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab")
psms = tables.spectrum_match_table
psms.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/tiago/.ppx/PRD000084/generated/PRIDE_Exp_Complete_Ac_15505.pride.mztab'

In [ ]:
tables.keys()

In [ ]:
prt = pd.DataFrame(tables['PRT'])

In [ ]:
prt.head()

In [ ]:
prt['description'].str.contains('unknown').sum() #Number of unknown proteins confirmed 

In [ ]:
prt["description"].str.contains("putative").sum() #Number of putative proteins 

In [ ]:
prt["description"].str.contains("hypothetical").sum() #Nuber of hypothetical proteins

In [ ]:
prt['description'].str.contains('lnc').sum() #Number of long non coding RNA?

In [ ]:
prt['description'].str.contains('coding', case=False).sum() #Number of coding RNA?

In [ ]:
prt['description'].str.contains('non', case=False).sum() #Number of coding RNA

In [ ]:
idx = prt['description'].str.contains('unknown') #see the proteins found
idx

In [ ]:
idx[0] = False
prt[idx]

In [ ]:
accession = list(set(accession))

print(len(accession))
accession[:10]

In [ ]:
ncbi = NCBITaxa()


tree = ncbi.get_topology(accession[:3])

print (tree.get_ascii(attributes=["sci_name", "rank"]))

# Not working, make plot of species available, can use other tools
# https://phylot.biobyte.de
ts = TreeStyle()
ts.show_leaf_name = True
ts.mode = "c"
ts.arc_start = -180 # 0 degrees = 3 o'clock
ts.arc_span = 180

for n in tree.traverse():
    if n.is_leaf():
        n.name = n.sci_name

tree.render("pride_plant_tree.pdf", w=500, tree_style=ts)